In [15]:
import numpy as np
import networkx as nx
import powerlaw 
import netgraph
import matplotlib.pylab as plt
from matplotlib.pyplot import savefig
import pandas as pd
import os
import pickle

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.samplers.regular.leiden_sampler import LeidenSampler
from Qommunity.samplers.regular.louvain_sampler import LouvainSampler
#from Qommunity.samplers.regular.bayan_sampler import BayanSampler

from Qommunity.searchers.community_searcher.community_searcher import CommunitySearcher
from Qommunity.searchers.hierarchical_community_searcher import HierarchicalCommunitySearcher

from iterative_searcher.iterative_searcher import IterativeSearcher

In [43]:
class GraphFromCSV:
    """
    It creates an object with different properties from a csv. Everything related to it, will be
        saved with the provided 'name' followed by the proper extensions.
    As of now, this is not the latest version of this class (or python script), but for this pipeline is enough.
    """

    def __init__(self, graph, name, base_dir='/'):
        self.graph = graph  # Graph file name
        self.conns = pd.read_csv(graph, delimiter=',', header=None).values # Graph connections
        self.graph_size = self.conns.shape
            
        self.name = name
        if base_dir == '/':
            self.dir = os.getcwd()+'/'
        else:
            self.dir = base_dir
        self.originals = self.conns
        # TODO: Add check path for base_dir!

    def __revert(self):
        """
        Reorders AAL3 regions by hemispheres.
        Odd indices correspond to Left hemisphere regions.
        Even indices correspond to rigth hemisphere regions.
        Stores a dictionary with the reodering of indices.
        """
        odd_odd = self.conns[::2, ::2]
        odd_even = self.conns[::2, 1::2]
        first = np.vstack((odd_odd, odd_even))
        even_odd = self.conns[1::2, ::2]
        even_even= self.conns[1::2, 1::2]
        second = np.vstack((even_odd, even_even))
        self.conns = np.hstack((first,second))

        # To map actual labels with original ones
        labels = np.array([x for x in range(0, self.graph_size[0])])
        left = np.array([x for x in range(1, self.graph_size[0], 2)])
        rigth = np.array([x for x in range(0, self.graph_size[0], 2)])
        self.hemis = dict(zip(labels, np.concatenate((left, rigth), axis=0)))

    def __take_log(self):
        """
        Takes the natural logarithm of the connections. Enhances visualisation of the matrix.
        """
        self.conns = np.log1p(self.conns)

    def __plot_graph(self, save=True, show=False, fig_size=(20,15), dpi=500):
        """
        Plot a graph. It assumes that the adjancency matrix is a csv file.
        """
        plt.figure(figsize=fig_size)
        plt.imshow(self.conns)
        cbar = plt.colorbar()
        cbar.set_label('Connection Strength', rotation=270)
        plt.tight_layout()
        if save:
            plt.savefig(self.dir+self.name+'.svg', format='svg', dpi=dpi)      
        if show:
            plt.show()     

    def process_graph(self, log=True, reshuffle=True, save=True, show=False, fig_size=(20,15)):
        """
        Applies default operations to the graph to work with it.
        """
        self.processed = True # The object has been processed
        if self.conns.shape[0] <= 1:
            raise ValueError("You are trying to process a flat graph. Can't do it. Unflatten your graph and set it to default.")
        else:
            if log:
                self.__take_log()
            if reshuffle:
                self.__revert()
            self.__plot_graph(save=save, show=show, fig_size=(20,15))
    
    def get_connections(self, ini=False):
        if not ini:
            return self.conns 
        else:
            return self.originals
    
    def flatten_graph(self, save=True):
        """
        Flatten the lower triangular adjancency matrix of the graph. 
        The flattened graph becomes available after applying this method.
        """
        x = self.conns.shape[0] # Dimensions of the graph 
        if x <= 1:
            raise ValueError("Dimension of the graph is 1 (or lower). You can't flattened an already flattened graph")
        else:
            dims = int(self.conns.shape[0]*(self.conns.shape[0]-1)/2)
            self.flat_conns = np.zeros((1,dims))
            k = 0
            for i in range(x):
                for j in range(i):
                    self.flat_conns[0,k] = self.conns[i,j]
                    k += 1
            if save:
                np.savetxt(self.dir+self.name+'_flatCM.csv', self.flat_conns, delimiter=',')
            return self.flat_conns

    def unflatten_graph(self, to_default=False):
        """
        Unflatten a graph and transform it to a square symmetric matrix. 
        The unflattened graph becomes available after applying this method.
        to_default: bool - The unflattened matrix becomes the default graph and replaces 
            the initial flat graph. As a checkpoint, the flattened graph is saved in the directory(default: False)
        """
        x = self.conns.shape[0] # First dimension of the flattened graph 
        flat_dim = self.conns.shape[1]
        if x > 1:
            raise ValueError("Dimension of the graph greater than 1. You can't unflattened an already unflattened graph")
        else:
            dims = int(1+np.sqrt(1+8*flat_dim)/2) # Dimensions of the squared graph
            self.unflat_conns = np.zeros((dims,dims))
            k = 0
            for i in range(dims):
                for j in range(i):
                    self.unflat_conns[i, j] = self.conns[0, k]
                    self.unflat_conns[j, i] = self.conns[0, k]
                    k += 1
            if to_default:
                # We save the flat graph with another name
                np.savetxt(self.dir+self.name+'_flatCM.csv', self.conns, delimiter=',')
                # We replace the original file with the unflattend graph
                np.savetxt(self.graph, self.unflat_conns, delimiter=',')
                # We re-initialize the graph with the unflattened graph and both the same name and directory
                self.__init__(self.graph, self.name, self.dir)
            return self.unflat_conns

In [80]:
G_process = GraphFromCSV("Brain-networks/control/ses-preop/sub-CON10_ses-preop_flatCM.csv", name="sub-CON10_ses-preop", base_dir="Brain-networks/")
network = G_process.unflatten_graph(to_default=False)
network = np.delete(network, [34,35,80,81], axis=0)
network = np.delete(network, [34,35,80,81], axis=1)
graph = nx.from_numpy_array(network, create_using=nx.Graph, edge_attr="weight")

In [113]:
num_runs = 20

In [83]:
resolution = 1
adv_sampler = AdvantageSampler(graph, resolution=resolution, num_reads=100, use_clique_embedding=True)
adv_searcher = HierarchicalCommunitySearcher(adv_sampler)
adv_iterative = IterativeSearcher(adv_searcher)

In [111]:
results_it = adv_iterative.run_with_full_sampleset_info(num_runs=20, save_results=False)

100%|██████████| 20/20 [03:39<00:00, 10.97s/it]


In [114]:
for i in range(20):
    print(results_it[i][-1])

[0.0, 0.3018141261362187, 0.3890632654692067]
[0.0, 0.27518833151663774, 0.35697531392051574]
[0.0, 0.27518833151663774, 0.35697531392051574]
[0.0, 0.3066523539528263, 0.39240421066131176]
[0.0, 0.31323015328158943, 0.3996874053186702]
[0.0, 0.29471137896395705, 0.38125512296834957]
[0.0, 0.27518833151663774, 0.35697531392051574]
[0.0, 0.29471137896395705, 0.38125512296834957]
[0.0, 0.27518833151663774, 0.35697531392051574]
[0.0, 0.3066523539528263, 0.39240421066131176]
[0.0, 0.27518833151663774, 0.35697531392051574]
[0.0, 0.30021144774515596, 0.388987014094897]
[0.0, 0.29471137896395705, 0.38125512296834957]
[0.0, 0.27518833151663774, 0.35697531392051574]
[0.0, 0.2873214865862525, 0.36944874762766533]
[0.0, 0.3133629616407433, 0.4072822976499915]
[0.0, 0.28218520273950903, 0.3685848905363981]
[0.0, 0.27990295771221096, 0.36474306769525244]
[0.0, 0.2873214865862525, 0.36944874762766533]
[0.0, 0.2744968163559066, 0.36364355213001154]


In [102]:
results = adv_searcher.hierarchical_community_search(division_tree=False, return_modularities=True, max_depth=None)

In [119]:
louv_sampler = LeidenSampler(graph)#, resolution=resolution)
louv_searcher = CommunitySearcher(louv_sampler)
louv_iterative = IterativeSearcher(louv_searcher)
cs_all, mod_all, ts_all = louv_iterative.run(num_runs=num_runs, save_results=False)

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 44.71it/s]


In [120]:
mod_all

array([0.43055321, 0.43055321, 0.43055321, 0.43055321, 0.43055321,
       0.43055321, 0.43055321, 0.43055321, 0.43055321, 0.43055321,
       0.43055321, 0.43055321, 0.43055321, 0.43055321, 0.43055321,
       0.43055321, 0.43055321, 0.43055321, 0.43055321, 0.43055321])

In [135]:
LC = nx.algorithms.community.louvain_communities(graph, weight='weight')
LM = nx.algorithms.community.modularity(graph, LC, weight='weight')

In [136]:
LP = nx.algorithms.community.louvain_partitions(graph, weight='weight')

In [147]:
LC[0]

{1,
 3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 69,
 71,
 73,
 75,
 77,
 108,
 117,
 119,
 121,
 123,
 125,
 127,
 128,
 129,
 131,
 133,
 135,
 137,
 139,
 141,
 143,
 145,
 147,
 153,
 155,
 157,
 159,
 160,
 161,
 163,
 164,
 165}

In [140]:
top = results_it[14]

In [145]:
top[0][0]

[1,
 5,
 7,
 9,
 12,
 13,
 15,
 33,
 34,
 35,
 37,
 39,
 41,
 44,
 45,
 46,
 47,
 49,
 50,
 51,
 53,
 55,
 57,
 59,
 61,
 63,
 65,
 67,
 68,
 69,
 71,
 75,
 77,
 78,
 79,
 80,
 81,
 84,
 85,
 89,
 119,
 123,
 125,
 127,
 135,
 137,
 139,
 141,
 143,
 145,
 157,
 159]

In [186]:
G = nx.barabasi_albert_graph(30, 2)
for (u,v,w) in G.edges(data=True):
    w['weight'] = 1000*np.random.rand()

In [187]:
resolution = 1
adv_sampler = AdvantageSampler(G, resolution=resolution, num_reads=100, use_clique_embedding=True)
adv_searcher = HierarchicalCommunitySearcher(adv_sampler)
adv_iterative = IterativeSearcher(adv_searcher)
results_weighted_adv = adv_iterative.run(num_runs=20, save_results=False)

100%|██████████| 20/20 [05:30<00:00, 16.54s/it]


In [188]:
results_weighted_adv[1]

array([0.37375834, 0.37375834, 0.37375834, 0.37375834, 0.37375834,
       0.37375834, 0.37375834, 0.37375834, 0.37375834, 0.37375834,
       0.37375834, 0.37375834, 0.37375834, 0.37375834, 0.37375834,
       0.37375834, 0.37375834, 0.37375834, 0.37375834, 0.37375834])

In [200]:
LC = nx.algorithms.community.louvain_communities(G, weight=None, resolution=resolution)
LM = nx.algorithms.community.modularity(G, LC, weight=None, resolution=resolution)

In [201]:
LM

0.3552295918367347

In [194]:
louv_sampler = LouvainSampler(G, resolution=resolution)
louv_searcher = CommunitySearcher(louv_sampler)
louv_iterative = IterativeSearcher(louv_searcher)
cs_all, mod_all, ts_all = louv_iterative.run(num_runs=num_runs, save_results=False)

100%|██████████| 20/20 [00:00<00:00, 335.68it/s]


In [195]:
mod_all

array([0.4632551 , 0.4632551 , 0.4632551 , 0.4632551 , 0.46625909,
       0.46820007, 0.46609982, 0.4632551 , 0.46820007, 0.46609982,
       0.4632551 , 0.46625909, 0.46820007, 0.4632551 , 0.46625909,
       0.46820007, 0.46820007, 0.46625909, 0.46820007, 0.4632551 ])